# Resume Outcome Predictor for Data Science - EDA

## 1. Inital Overview of the Dataset

- 1000 instances
- 11 different features
- About half object types and half integers
- Target variable will be Recruiter Decision

[See the Dataset](https://www.kaggle.com/datasets/mdtalhask/ai-powered-resume-screening-dataset-2025?resource=download)

In [2]:
import sys
sys.path.append('../')

from src.data_load import extract_zip, load_csv

extract_zip()
df = load_csv()

print("Head of dataset")
print(df.head(), "\n")
print("Dataset info")
print(df.info(), "\n")
print("Dataset shape")
print(df.shape, "\n")
print("Dataset columns")
print(df.columns, "\n")

Extracted '/Users/ehagye/Documents/ResumeProject/ResumeRanker/data/raw/ResumeData.zip' to '/Users/ehagye/Documents/ResumeProject/ResumeRanker/data/interim'
Head of dataset
   Resume_ID              Name                                        Skills  \
0          1        Ashley Ali                      TensorFlow, NLP, Pytorch   
1          2      Wesley Roman  Deep Learning, Machine Learning, Python, SQL   
2          3     Corey Sanchez         Ethical Hacking, Cybersecurity, Linux   
3          4  Elizabeth Carney                   Python, Pytorch, TensorFlow   
4          5        Julie Hill                              SQL, React, Java   

   Experience (Years) Education                Certifications  \
0                  10      B.Sc                           NaN   
1                  10       MBA                     Google ML   
2                   1       MBA  Deep Learning Specialization   
3                   7    B.Tech                 AWS Certified   
4                   4 

## 2. Check for Missing Values
- Only missing values for certification feature
- This means that the job applicant does not have a certificate so they won't be removed


In [3]:
# printing total null of each feature
print("Amount of Null for each feature")
print(df.isnull().sum())

# printing percentage of null for each feature
print("\n", "Percentage of Null for each feature")
df.isnull().mean() * 100

Amount of Null for each feature
Resume_ID                   0
Name                        0
Skills                      0
Experience (Years)          0
Education                   0
Certifications            274
Job Role                    0
Recruiter Decision          0
Salary Expectation ($)      0
Projects Count              0
AI Score (0-100)            0
dtype: int64

 Percentage of Null for each feature


Resume_ID                  0.0
Name                       0.0
Skills                     0.0
Experience (Years)         0.0
Education                  0.0
Certifications            27.4
Job Role                   0.0
Recruiter Decision         0.0
Salary Expectation ($)     0.0
Projects Count             0.0
AI Score (0-100)           0.0
dtype: float64

## 3. Examine Categorical Features
- There is an even distribution of categories for almost all features
- However the target variable *Recruiter Decision* is very imbalanced with majority Hire
- This is surprising as you would assume most Resumes to be rejected
- Will want to create more data points to even the balance maybe using SMOTE

In [4]:
# printing out each categorical feature and their amounts
print(df['Education'].value_counts(), "\n")
print(df['Certifications'].value_counts(), "\n")
print(df['Job Role'].value_counts(), "\n")
print(df['Recruiter Decision'].value_counts(), "\n")
print(df['Skills'].str.split(', ').explode().value_counts())


Education
B.Sc      205
MBA       202
B.Tech    200
M.Tech    198
PhD       195
Name: count, dtype: int64 

Certifications
Deep Learning Specialization    255
AWS Certified                   249
Google ML                       222
Name: count, dtype: int64 

Job Role
AI Researcher            257
Data Scientist           255
Cybersecurity Analyst    255
Software Engineer        233
Name: count, dtype: int64 

Recruiter Decision
Hire      812
Reject    188
Name: count, dtype: int64 

Skills
Python              388
SQL                 370
Ethical Hacking     206
TensorFlow          205
Machine Learning    200
NLP                 195
Linux               191
Pytorch             189
Java                188
Deep Learning       184
Cybersecurity       184
Networking          184
C++                 168
React               165
Name: count, dtype: int64


## 4. Examine Numberic Features